In [ ]:
# | default_exp nets/symswin_3d

### Inspiration

This architecture is only an idea. It builds on SwinV2, however, instead of applying windowed attention in a window of consecutive tokens, it applies attention on symmetrically opposite tokens (along the x-axis). This was inspired from medical radiology imaging, particularly HeadCTs, that rely heavily on symmetry in the left and right hemispheres to determine abnormality.

### Logic

# Imports

In [ ]:
# | export


import torch
from einops import rearrange

In [ ]:
# | export


def symmetry_attention_rearrange_forward(hidden_states: torch.Tensor):
    # hidden_states: (b, num_patches_z, num_patches_y, num_patches_x, dim)

    # Flip the second half of the patches along the x axis
    last_dim_length = hidden_states.shape[-2]
    hidden_states = torch.cat(
        [
            hidden_states[..., : last_dim_length // 2, :],
            hidden_states[..., last_dim_length // 2 :, :].flip(-2),
        ],
        dim=-2,
    )
    # (b, num_patches_z, num_patches_y, num_patches_x_first_half + flipped_num_patches_second_half, dim)

    # Rearrange the patches to alternate between the two halves along the x axis
    hidden_states = rearrange(
        hidden_states,
        "b num_patches_z num_patches_y (two half_num_patches_x) dim -> "
        "b num_patches_z num_patches_y (half_num_patches_x two) dim",
        two=2,
    ).contiguous()
    # (b, num_patches_z, num_patches_y, rearranged_num_patches_x, dim)

    return hidden_states


def symmetry_attention_rearrange_backward(hidden_states: torch.Tensor):
    # hidden_states: (b, num_patches_z, num_patches_y, rearranged_num_patches_x, dim)

    # Return the patches to their previous order along the x-axis
    hidden_states = rearrange(
        hidden_states,
        "b num_patches_z num_patches_y (half_num_patches_x two) dim -> "
        "b num_patches_z num_patches_y (two half_num_patches_x) dim",
        two=2,
    ).contiguous()
    # (b, num_patches_z, num_patches_y, num_patches_x_first_half + flipped_num_patches_second_half, dim)

    # Flip the second half of the patches along the x axis to return to the original state
    last_dim_length = hidden_states.shape[-2]
    hidden_states = torch.cat(
        [
            hidden_states[..., : last_dim_length // 2, :],
            hidden_states[..., last_dim_length // 2 :, :].flip(-2),
        ],
        dim=-2,
    )
    # (b, num_patches_z, num_patches_y, num_patches_x, dim)

    return hidden_states

In [ ]:
test_arr = torch.arange(64).reshape(1, 1, 8, 8, 1)

forward_test_arr = symmetry_attention_rearrange_forward(test_arr)
backward_forward_test_arr = symmetry_attention_rearrange_backward(forward_test_arr)

assert torch.allclose(test_arr, backward_forward_test_arr)

test_arr.squeeze(0, 1, -1), forward_test_arr.squeeze(0, 1, -1), backward_forward_test_arr.squeeze(0, 1, -1)


(
    tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 8,  9, 10, 11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29, 30, 31],
        [32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47],
        [48, 49, 50, 51, 52, 53, 54, 55],
        [56, 57, 58, 59, 60, 61, 62, 63]]),
    tensor([[ 0,  7,  1,  6,  2,  5,  3,  4],
        [ 8, 15,  9, 14, 10, 13, 11, 12],
        [16, 23, 17, 22, 18, 21, 19, 20],
        [24, 31, 25, 30, 26, 29, 27, 28],
        [32, 39, 33, 38, 34, 37, 35, 36],
        [40, 47, 41, 46, 42, 45, 43, 44],
        [48, 55, 49, 54, 50, 53, 51, 52],
        [56, 63, 57, 62, 58, 61, 59, 60]]),
    tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 8,  9, 10, 11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29, 30, 31],
        [32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47],
        [48, 49, 50, 51, 52, 53, 54, 55],
        [56, 57

# nbdev

In [ ]:
!nbdev_export